In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
import theano 
import tensorflow.keras as keras
import tensorflow as tf 



In [2]:
#NativeUTRs=pd.read_csv("/Users/harrys/Downloads/Supplemental_code/Data/Native_UTRs.csv")
RandUTRs=pd.read_csv("/Users/harrys/Downloads/Supplemental_code/Data/Random_UTRs.csv").head(1000)
#NativeUTRs.head()
RandUTRs.head()

,Unnamed: 0,UTR,growth_rate,t0,t1
0,0,AAAAAAAAAACATAATAACGATGATCAGTTAAAATCATAGTCTAAG...,-1.237065,14,3
1,1,AAAAAAAAAAGACTACAACAGATTGTAGTGGCGGACCAGTGTGCCT...,1.288663,14,49
2,2,AAAAAAAAAATATGGGGCCCTGTTCCAAAGATACCTCAATTTCATA...,-0.608457,13,6
3,3,AAAAAAAAAATCTCTGGCCCGATTATACTGGAGCTAATGTAAAATT...,-1.093964,12,3
4,4,AAAAAAAAACATAAATATGAAGGCCTGACATTATAAATAACTTACC...,-0.048841,7,6


In [46]:
def one_hot_encoding(df, seq_column, expression):
    
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4))) # {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}

    n = len(df)
    
    # length of the UTR sequence
    # we also add 10 empty spaces to either side
    total_width = df[seq_column].str.len().max() + 20
    
    # initialize an empty numpy ndarray of the appropriate size
    X = np.zeros((n, 1, 4, total_width))
    
    # an array with the sequences that we will one-hot encode
    seqs = df[seq_column].values
    
    # loop through the array of sequences to create an array that keras will actually read
    for i in range(n):
        seq = seqs[i]
        
        # loop through each individual sequence, from the 5' to 3' end
        for b in range(len(seq)):
            # this will assign a 1 to the appropriate base and position for this UTR sequence
            X[i, 0, base_dict[seq[b]], int(b + round((total_width - len(seq))/2.))] = 1.
    
        # keep track of where we are
        if (i%10000)==0:
            print(str(i)+",")
        
    #X = X.astype(theano.config.floatX)
    X = X.astype('float32')
    Y = np.asarray(df[expression].values,
                   dtype = 'float32')[:, np.newaxis]
    
    return X, Y, total_width
X,Y,total_width = one_hot_encoding(RandUTRs,"UTR","growth_rate")

0,


In [47]:
#dict(zip(['A','C','G','T'],range(4)))
#      ,,# of sequences = 1000,total_width = 70 
#np.zeros((10,1,4,6))
    #why have "1" as second dimension 
#RandUTRs["UTR"][0]

#Z= np.zeros((5,4,70))
#Z[0,1,]

#b+(70-50)/2
#help(X.astype)
#z= tf.keras.backend.cast_to_floatx(X)
#type(z[0][0][0][0])
print(type(X[0][0][0][0]))

#X=tf.dtypes.cast(X, tf.float64)
type(Y[0])
#theano.config.floatX


<class 'numpy.float32'>


numpy.ndarray

In [12]:
import random 
# a sorted numpy array of UTR indexes, from least reads to most reads
sorted_inds = RandUTRs.sort_values('t0').index.values


train_inds = sorted_inds[:int(0.95*len(sorted_inds))] # 95% of the data as the training set


test_inds = sorted_inds[int(0.95*len(sorted_inds)):] # UTRs with most reads at time point 0 as the test set

# set the seed before randomly shuffling the data
seed = 0.5
random.shuffle(train_inds, lambda :seed)

In [7]:
space = {   'conv_width': hp.choice('conv_width', [9, 13, 17, 25]), #
            'conv_filters': hp.choice('conv_filters', [32, 64, 128, 256]), #
            'conv_layers': hp.choice('conv_layers', [2, 3, 4]), #
            'dense_layers': hp.choice('dense_layers', [1, 2]), #
            'conv_dropout': hp.choice('conv_dropout',  [None, 0.15]), # 
            'dense_dropout': hp.choice('dense_dropout', [None, 0.1, 0.25, 0.5]), 
            'dense_units': hp.choice('dense_units', [32, 64, 128, 256]), #
        }

NameError: name 'hp' is not defined

In [ ]:
def create_model(params):
    model = Sequential()
    model.add(Convolution2D(params['conv_filters'],
                            4,
                            params['conv_width'],
                            border_mode = 'valid',
                            input_shape = (1, 4, total_width),
                            activation = 'relu'))
    
    # add dropout at the convolutional layers if appropriate
    if params['conv_dropout']:
        model.add(Dropout(p = params['conv_dropout']))
    
    # add the appropriate number of additional convolutional layers, along with dropout
    for i in range(params['conv_layers'] - 1):
        model.add(Convolution2D(params['conv_filters'],
                                1,
                                params['conv_width'],
                                border_mode = 'same',
                                activation = 'relu'))
        
        if params['conv_dropout']:
            model.add(Dropout(params['conv_dropout']))
    
    model.add(Flatten())
    
    # add the appropriate number of dense layers and dropout
    for i in range(params['dense_layers']):
        model.add(Dense(output_dim = params['dense_units'],
                        activation = 'relu'))
        
        if params['dense_dropout']:
            model.add(Dropout(p = params['dense_dropout']))
    
    # add the output layer, since we want to predict the "growth rate" we only want a single 
    # number, hence the single dimension
    model.add(Dense(output_dim = 1))
    
    # compile the model
    model.compile(loss = 'mean_squared_error',
                  optimizer = 'adam',
                  metrics = ['mean_squared_error'])
    
    return model